In [8]:
import json
import os
from unidecode import unidecode
import re

input_dir = '../gym/json/'


### JSON example
# {
#     "PAPER'S NUMBER OF TABLES": 5,   to be removed
#     "S1.T1": {                       the table_id
#         "caption": "Table 1. Existing",
#         "table": "<table id=\"S1.T1.1.1\" +</tr>\n</table>\n\n",
#         "footnotes": [
#             "1"
#         ],
#         "references": [
#             "A",
#         ],
#     },
# }

def read_json(file):
    with open(file, 'r') as f:
        data = json.load(f)
    return data


def clean_text(text):
    #cast to string
    text = str(text)
    text = text.strip()
    # Remove HTML tags and attributes
    text = re.sub(r'<[^>]*>', ' ', text)
    # De-accent
    text = unidecode(text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)

    # Remove multiple spaces and newlines
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n+', ' ', text)

    return text

In [9]:
# Creating a table with table_id, caption, footnotes, references

import pandas as pd

df = pd.DataFrame(
    columns=[
        'id',
        'file_id',
        'table_id',
        'table_body',
        'caption',
        'references'
    ]
)

for file in os.listdir(input_dir):
    data = read_json(input_dir + file)
    for table_id in data:
        file_id = file.split('.')[0] + '.' + file.split('.')[1]
        table = data[table_id]
        table_body = clean_text(table['table'])
        caption = clean_text(table['caption'])
        references = clean_text(' '.join(table['references']))
        df.loc[len(df)] = [len(df), file_id, table_id, table_body, caption, references]

df.head()

,id,file_id,table_id,table_body,caption,references
0,0,1609.05600v2,S5.T1,Avg score Avg accuracy Method over scenes ove...,Table 1 Results on the test set of the balance...,We compare our method against the three models...
1,1,1609.05600v2,S5.T2,Multiple choice Open ended Method Overall Yes...,Table 2 Results on the test set of the abstrac...,We report our results on the original abstract...
2,2,2006.05148,S3.T1,Notation Meaning U U mathcal U of devices u i...,Table 1 List of Notations,
3,3,2006.05148,S4.T2,Method 1 Target Label N 1 N 1 N 1 N 2 N 2 N 2...,Table 2 Test accuracy evaluation for different...,
4,4,2006.05148,S4.T3,Number of Averaging Samples p p p p 1 p 1 p 1...,Table 3 MDS comparison mean standard deviation...,


In [10]:
df.to_csv('../gym/data.csv')